<div style="background-color: #c3e8fb; padding: 10px; color: #144d84;">
<b>Exercise 2) Model Subclassing for Persian Number Classification</b><br>
Write a convolutional neural network for classifying Persian numbers using batch normalization and residual connections. This network must use model subclassing, and its accuracy on the evaluation dataset should not be below 98%.
</div>

In [92]:
import matplotlib.pyplot as plt

# cv2 and io for loading hoda
import cv2
from scipy import io

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Add

In [93]:
def load_hoda(training_sample_size=1000, test_sample_size=200, size=5):
    #load dataset
    trs = training_sample_size
    tes = test_sample_size
    dataset = io.loadmat('Data_hoda_full.mat')

    #test and training set
    X_train_orginal = np.squeeze(dataset['Data'][:trs])
    y_train = np.squeeze(dataset['labels'][:trs])
    X_test_original = np.squeeze(dataset['Data'][trs:trs+tes])
    y_test = np.squeeze(dataset['labels'][trs:trs+tes])

    #resize
    X_train_5by5 = [cv2.resize(img, dsize=(size, size)) for img in X_train_orginal]
    X_test_5by_5 = [cv2.resize(img, dsize=(size, size)) for img in X_test_original]
    #reshape
    X_train = np.reshape(X_train_5by5, [-1,size**2])
    X_test = np.reshape(X_test_5by_5, [-1,size**2])

    return X_train, y_train, X_test, y_test

In [94]:
hw = 30
x_train, y_train, x_test, y_test = load_hoda(training_sample_size=1000, test_sample_size=200, size=hw)

In [95]:
x_train= x_train.reshape(-1, hw, hw, 1)
x_test= x_test.reshape(-1, hw, hw, 1)

In [96]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

print(x_train.shape)

(1000, 30, 30, 1)


In [97]:
class HodaModel(Model):
    def __init__(self):
        super(HodaModel, self).__init__()
        self.conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')
        self.bn1 = BatchNormalization()

        self.conv2 = Conv2D(32, (3, 3), activation=None, padding='same')
        self.bn2 = BatchNormalization()

        self.conv3 = Conv2D(64, (3, 3), activation='relu', padding='same')
        self.bn3 = BatchNormalization()

        self.conv4 = Conv2D(64, (3, 3), activation=None, padding='same')
        self.bn4 = BatchNormalization()

        self.max_pool = MaxPooling2D((2, 2))
        self.flatten = Flatten()
        self.dense1 = Dense(128, activation='relu')
        self.dense2 = Dense(10, activation='softmax')

    def residual_block(self, x, conv, bn, filters):
        y = conv(x)
        y = bn(y)
        y = tf.nn.relu(y)
        return y + x

    def call(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = tf.nn.relu(x)

        # residual connection
        x = self.residual_block(x, self.conv2, self.bn2, 32)

        x = self.max_pool(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = tf.nn.relu(x)

        # residual connection
        x = self.residual_block(x, self.conv4, self.bn4, 64)

        x = self.max_pool(x)

        x = self.flatten(x)
        x = self.dense1(x)
        x = tf.nn.relu(x)
        x = self.dense2(x)

        return x

model = HodaModel()


In [98]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=20, batch_size=128, validation_data=(x_test, y_test),
          callbacks=[tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, verbose=1)])

Epoch 1/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 315ms/step - accuracy: 0.3272 - loss: 1.9518 - val_accuracy: 0.8300 - val_loss: 0.5583
Epoch 2/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8269 - loss: 0.5566 - val_accuracy: 0.8500 - val_loss: 0.4587
Epoch 3/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8971 - loss: 0.3452 - val_accuracy: 0.9250 - val_loss: 0.2388
Epoch 4/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9318 - loss: 0.2235 - val_accuracy: 0.9650 - val_loss: 0.1889
Epoch 5/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9494 - loss: 0.1449 - val_accuracy: 0.9550 - val_loss: 0.1411
Epoch 6/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9599 - loss: 0.1144 - val_accuracy: 0.9650 - val_loss: 0.1439
Epoch 7/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9734 - loss: 0.0698 - val_accuracy: 0.9700 - val_loss: 0.1307
Epoch 8/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9848 - loss: 0.0485 - val_accuracy: 0.9800 - val_loss: 0.1253

In [100]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'acc on test data: {test_acc:.2f}')

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9779 - loss: 0.1178 
acc on test data: 0.98
